# 유튜브 댓글 크롤링 (1)
### keyword: 보안쟁이

In [9]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.parse import quote
from tqdm import tqdm

In [10]:
KEYWORD = "보안쟁이"

In [11]:
v_url_list = []
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
url = 'https://www.youtube.com/results?search_query=' + quote(KEYWORD)
driver.get(url)
time.sleep(2)

# channel_url, channel_id, channel_name 가져오기
main_link = driver.find_element_by_id("main-link")
channel_url = main_link.get_attribute("href")
c_idx = channel_url.rfind("/") + 1
channel_id = channel_url[c_idx:]
channel_name = main_link.find_element_by_id("text-container").text

# 채널 내 '동영상' 으로 이동
main_link.click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="tabsContent"]/tp-yt-paper-tab[2]/div').click()
time.sleep(1)

# 스크롤 끝까지 내리기
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
driver.execute_script("window.scrollTo(0, 350);")
while True: 
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
    time.sleep(3) 
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight") 
    if new_page_height == last_page_height: 
        break 
    last_page_height = new_page_height

lis = driver.find_elements_by_id('thumbnail')

for li in lis:
    video_url = li.get_attribute("href")
    v_url_list.append(video_url)

driver.close()

In [12]:
print(len(v_url_list), "\n", v_url_list)

146 
 ['https://www.youtube.com/watch?v=4ghRdR2UoLs', 'https://www.youtube.com/watch?v=H9fJ1ycjzWk', 'https://www.youtube.com/watch?v=P75YHMkdMN8', 'https://www.youtube.com/watch?v=FjBikdRCd3M', 'https://www.youtube.com/watch?v=P0PYGk93yKk', 'https://www.youtube.com/watch?v=XvIQgQeN0_I', 'https://www.youtube.com/watch?v=INLNzjk8akE', 'https://www.youtube.com/watch?v=FKjzVEquUWY', 'https://www.youtube.com/watch?v=zaaXR61r-ng', 'https://www.youtube.com/watch?v=Y4nXATT-1ys', 'https://www.youtube.com/watch?v=_lfglwY__20', 'https://www.youtube.com/watch?v=RRDK9F6wiK4', 'https://www.youtube.com/watch?v=VYrZdO0Qxls', 'https://www.youtube.com/watch?v=QxVpXSjNNO4', 'https://www.youtube.com/watch?v=7FrEebGYs9U', 'https://www.youtube.com/watch?v=Z82p2qITQDE', 'https://www.youtube.com/watch?v=G5iZIX_jckk', 'https://www.youtube.com/watch?v=QU6q0cljdYs', 'https://www.youtube.com/watch?v=-FXrIDQKoEg', 'https://www.youtube.com/watch?v=7wXfkWJndKI', 'https://www.youtube.com/watch?v=ebOGw7xKMko', 'https

In [14]:
comment_list = []; err = 0
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
for url in tqdm(v_url_list):
    if url == None:
        continue
    else:
        driver.get(url)
        time.sleep(1)

        # 스크롤 끝까지 내리기
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script("window.scrollTo(0, 350);")
        time.sleep(2)
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(1.5)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # 댓글 수집
        contents = driver.find_element_by_class_name("style-scope ytd-item-section-renderer")
        try:
            comments = contents.find_elements_by_tag_name('ytd-expander')
            for comment in comments:
                text = comment.find_element_by_tag_name('div').text
                comment_list.append(text)
        except:
            err += 1
            continue
print(f'Error Count: {err}')
driver.close()

100%|██████████| 146/146 [32:32<00:00, 13.37s/it]
Error Count: 0


In [15]:
len(comment_list)

5123

### 댓글 원본 저장(전처리 X)

In [16]:
import pandas as pd 
df = pd.DataFrame({
    'comments': comment_list
})
df.head()

,comments
0,"1. KISA 인터넷 보호나라\n\n해킹/바이러스, 불법스팸이나 개인정보 침해 등으..."
1,좋은 영상 감사합니다. 문제가 생겼을때는 전문가의 도움을 받는게 좋군요!
2,새해 복 많이 받으세요~!
3,카페에 질문 남거도 답변이 없는데 연락할 방법이 없나요..?ㅠㅠ
4,카트 하다가 어떤핵쟁이가 아이피 따고 다른방에 제 아이피를 여기저기 뿌리고 다녀서....


In [17]:
df.to_csv(f'./data/youtube_channel={KEYWORD}_org.txt', sep='\t', index=False)

### 데이터 전처리

In [18]:
df_test = pd.read_table(f'./data/youtube_channel={KEYWORD}_org.txt')
df_test.head()

,comments
0,"1. KISA 인터넷 보호나라\n\n해킹/바이러스, 불법스팸이나 개인정보 침해 등으..."
1,좋은 영상 감사합니다. 문제가 생겼을때는 전문가의 도움을 받는게 좋군요!
2,새해 복 많이 받으세요~!
3,카페에 질문 남거도 답변이 없는데 연락할 방법이 없나요..?ㅠㅠ
4,카트 하다가 어떤핵쟁이가 아이피 따고 다른방에 제 아이피를 여기저기 뿌리고 다녀서....


In [19]:
df_test.shape

(5123, 1)

In [20]:
df_test['comments'] = df_test['comments'].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df_test.head()

<ipython-input-20-4884171cc048>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['comments'] = df_test['comments'].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,comments
0,KISA 인터넷 보호나라해킹바이러스 불법스팸이나 개인정보 침해 등으로 불편을 겪고...
1,좋은 영상 감사합니다 문제가 생겼을때는 전문가의 도움을 받는게 좋군요
2,새해 복 많이 받으세요
3,카페에 질문 남거도 답변이 없는데 연락할 방법이 없나요ㅠㅠ
4,카트 하다가 어떤핵쟁이가 아이피 따고 다른방에 제 아이피를 여기저기 뿌리고 다녀서


In [21]:
# 'comments' 값 공백 여부 확인
drop_idx = df_test[df_test.comments == ""].index
print(len(drop_idx), drop_idx)

20 Int64Index([  69,  784,  795,  916, 1084, 1109, 1118, 1493, 1537, 1600, 1884,
            2303, 2400, 3213, 3233, 3958, 4553, 4586, 4622, 5051],
           dtype='int64')


In [22]:
# 공백 값을 가지고 있는 해당 인덱스 행 제거
df_test.drop(drop_idx, inplace=True)
df_test.shape

(5103, 1)

In [23]:
# null값 여부 확인
df_test.isnull().sum()

comments    1
dtype: int64

In [24]:
# null 값을 포함한 행 제거
df_test = df_test.dropna(how='any')
df_test.shape

(5102, 1)

In [25]:
df_test.head()

,comments
0,KISA 인터넷 보호나라해킹바이러스 불법스팸이나 개인정보 침해 등으로 불편을 겪고...
1,좋은 영상 감사합니다 문제가 생겼을때는 전문가의 도움을 받는게 좋군요
2,새해 복 많이 받으세요
3,카페에 질문 남거도 답변이 없는데 연락할 방법이 없나요ㅠㅠ
4,카트 하다가 어떤핵쟁이가 아이피 따고 다른방에 제 아이피를 여기저기 뿌리고 다녀서


In [26]:
print(f'전처리 후 샘플의 개수: {len(df_test)}')

전처리 후 샘플의 개수: 5102


In [27]:
df_test.to_csv(f'./result/youtube_channel={KEYWORD}_res.txt', sep='\t', index=False)